<a href="https://colab.research.google.com/github/sheelapravalika/AI_CS_sheelapravalika13gmail/blob/main/ANN_47_(2)_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sheelapravalika/AI_CS_mokshasaireddy005/blob/main/ANN_47(2).ipynb)


In [ ]:
# STEP 1: Install required packages
!pip install -U datasets huggingface_hub fsspec pandas scikit-learn matplotlib seaborn numpy tensorflow

  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)


In [ ]:
# STEP 2: Import Libraries
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# STEP 3: Load CIC-IDS2017 Dataset (Binary)
dataset = load_dataset("sonnh-tech1/cic-ids-2017", "binary")
df = dataset["train"].to_pandas()

print(df)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/92.8M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2522362 [00:00<?, ? examples/s]

         DestinationPort  FlowDuration  TotalFwdPackets  TotalBackwardPackets  \
0                  54865             3                2                     0   
1                  55054           109                1                     1   
2                  55055            52                1                     1   
3                  46236            34                1                     1   
4                  54863             3                2                     0   
...                  ...           ...              ...                   ...   
2522357               53         32215                4                     2   
2522358               53           324                2                     2   
2522359            58030            82                2                     1   
2522360               53       1048635                6                     2   
2522361               53         94939                4                     2   

         TotalLengthofFwdPa

In [ ]:
# STEP 4: Preprocess Data
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])  # Benign=0, Threat=1

X = df.drop(['Label', 'LabelMap'], axis=1)
y = df['Label']

X_numeric = X.select_dtypes(include=['int64', 'float64'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

In [ ]:
# STEP 5: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [ ]:
# STEP 6: Build the ANN Model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# STEP 7: Train the Model
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=256,
                    callbacks=[early_stop],
                    verbose=1)

Epoch 1/10
5518/5518 ━━━━━━━━━━━━━━━━━━━━ 39s 7ms/step - accuracy: 0.4513 - loss: -1365857.0000 - val_accuracy: 0.4763 - val_loss: -16814986.0000
Epoch 2/10
5518/5518 ━━━━━━━━━━━━━━━━━━━━ 43s 7ms/step - accuracy: 0.4553 - loss: -30624836.0000 - val_accuracy: 0.4737 - val_loss: -91523048.0000
Epoch 3/10
5518/5518 ━━━━━━━━━━━━━━━━━━━━ 38s 6ms/step - accuracy: 0.4548 - loss: -125041776.0000 - val_accuracy: 0.4761 - val_loss: -252823344.0000
Epoch 4/10
5518/5518 ━━━━━━━━━━━━━━━━━━━━ 43s 7ms/step - accuracy: 0.4549 - loss: -312226112.0000 - val_accuracy: 0.4768 - val_loss: -529051264.0000
Epoch 5/10
5518/5518 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - accuracy: 0.4552 - loss: -622039488.0000 - val_accuracy: 0.4767 - val_loss: -947056576.0000
Epoch 6/10
5518/5518 ━━━━━━━━━━━━━━━━━━━━ 37s 7ms/step - accuracy: 0.4558 - loss: -1074278912.0000 - val_accuracy: 0.4762 - val_loss: -1535334784.0000
Epoch 7/10
5518/5518 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.4565 - loss: -1706015104.0000 - val_accu

In [ ]:
# STEP 8: Evaluate the Model
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32").flatten()

# Accuracy
print("✅ Accuracy:", accuracy_score(y_test, y_pred))

# Classification Report
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='coolwarm')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - ANN')
plt.show()

In [ ]:
# ✅ STEP 11: Plot Training History
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('ANN Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('ANN Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import nbformat
import os

# Get the path of the current notebook
notebook_path = os.path.abspath("__main__")

# Load and clean the notebook
try:
    with open(notebook_path, "r", encoding="utf-8") as f:
        nb = nbformat.read(f, as_version=4)

    # Remove problematic metadata
    for cell in nb.cells:
        if 'metadata' in cell and 'widgets' in cell['metadata']:
            del cell['metadata']['widgets']

    # Save cleaned notebook (you might want to save it with a different name)
    cleaned_notebook_path = "ANN_cleaned.ipynb"
    with open(cleaned_notebook_path, "w", encoding="utf-8") as f:
        nbformat.write(nb, f)
    print(f"Cleaned notebook saved to {cleaned_notebook_path}")

except FileNotFoundError:
    print(f"Error: The notebook file '{notebook_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: The notebook file '/content/__main__' was not found.
